# 第4章 データ前処理　よりよいトレーニングセットの構築

## 4.1 欠測データへの対処
* _欠測値(missing value)_はNaNやNULLとみなされる
* ほとんどの計算ツールは欠測値に対処できないか、欠測地を無視した場合に予期せぬ結果を生み出す。

### 4.1.1 欠測値の特定

In [1]:
import pandas as pd
from io import StringIO
# sample dataの作成
csv_data = '''A,B,C,D
             1.0,2.0,3.0,4.0
             5.0,6.0,,8.0
             10.0,11.0,12.0'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


欠測値を手動で探すのは大変。  
_**isnull()**_メソッドを使う。　

In [2]:
df.isnull() # セルにNaNがある場合はTrue

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [3]:
df.isnull().sum() # 列ごとのNaNの数をカウントする

A    0
B    0
C    1
D    1
dtype: int64

### 4.1.2 欠測値を持つサンプル/特徴量を取り除く
_**dropna()**_メソッドで欠測値を含んでいる行を削除

In [4]:
# 欠測値を含む行を削除
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [5]:
# axis=1
# 欠測値を含む列を削除
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [6]:
# すべての列がNaNである行だけを削除
# すべての列がNaNである行は存在しないのでそのまま返ってくる
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
# 非NaN値が4つ未満の行を削除
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# 特定の列にNaNが含まれている行だけを削除
# CにおいてNaNがある行が削除される
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 4.1.3 欠測値を補完する
dropna()で削除しすぎるとサンプル数が減り、解析の信頼性が失われることがある。  
**補間法(interpolation technique)**によって_データセットの他のトレーニングサンプルから欠測値を推測できる。_  
  
_**平均値補間(mean imputation)**_：欠測値をその列全体の平均値と置き換える。  
scikit-learn のImputerクラスを使用すると便利。

In [10]:
from sklearn.preprocessing import Imputer
# 欠測値補完のインスタンスを生成(平均値補完)
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
# データを適合
# df.valuesによってnumpy配列にアクセスできる
# skleanにはnp.array形式で投げること。
# DataFrameでは不可。
imr = imr.fit(df.values)
#補完を実行
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

* strategy引数のその他例
    * median(中央値)
    * most_frequent(最頻値)
* axis=0 を axis=1 にすると、行の平均値が計算されるようになる。

### 4.1.4 scikit-learn の推定器API  
  
* scikit-learnは_**変換器(transformer)**_クラスが存在する


* Imputerクラスもそのひとつ


* 基本的なメソッドにはfitとtransformの2つがある。  


* _**fitメソッド**_
    * トレーニングセットからパラメータを学習するために使用    


* _**transformメソッド**_
    * 学習したパラメータに基づいてデータを変換するために使用
    * fitした際の特徴量と同じにすること

* 第3章の分類器は_**推定器(estimator)**_に属している
* fitで学習し、predictメソッドで推定していたが、transformメソッドも使用できる

## 4.2 カテゴリデータの処理
数値ではなくカテゴリ値の特徴量の扱い方を理解する。

### 4.2.1 名義特徴量と順序特徴量
* 順序(ordinal)特徴量
    * 並び替えや順序付けが可能なカテゴリ値
        * Tシャツのサイズ: XL > L > M
* 名義(nominal)特徴量
    * 順序がない
        * Tシャルの色: 赤、青、黄

In [10]:
# サンプルデータセットの作成
# Tシャツの色、サイズ、価格、クラスラベル
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', '10.1', 'class1'],
    ['red', 'L', '13.5', 'class2'],
    ['blue', 'XL', '15.3', 'class1']
])

#列名を設定
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


[color] 名義特徴量  
[size]順序特徴量  
[price] 数値特徴量

### 4.2.2 順序特徴量のマッピング
順序特徴量の整数への変換マッピングは明示的に定義しなければならない。  
**<定義>**  
**_XL = L + 1 = M + 2_**


In [11]:
# Tシャツのサイズと整数を対応させるディクショナリを生成
size_mapping = {'XL': 3, 'L':2, 'M':1}
# Tシャツのサイズを整数に変換
df['size'] = df['size'].map(size_mapping) # これすげぇ便利
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [13]:
# 逆のマッピングもできる
# 整数値から元に戻す
inv_size_mapping = {v: k for k, v in size_mapping.items()} # なにをやっているのか謎
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 4.2.3 クラスラベルのエンコーディング
* クラスラベルはscikit-learnに投げる前に整数の配列としておくことで技術的なミスを回避する。
* クラスラベルが順序特徴量ではないことと、文字列のラベルにどの整数に割り当てるかは重要ではない。

In [14]:
import numpy as np
# クラスラベルと整数を対応させるディクショナリを作成
# 順序のときと同じ
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping # マッピングディクショナリ

{'class1': 0, 'class2': 1}

In [15]:
# マッピングディクショナリを使ってクラスラベルを整数に変換
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [16]:
# もとに戻すには辞書のキーと値を逆にすればよい
inv_class_mapping = {v:k for k, v in class_mapping.items()}
# 整数からクラスラベルに変換
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [18]:
# scikit-learnで直接実装されているLabelEncoderで変換する方法もある
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーダのインスタンスを作成
class_le = LabelEncoder()
# クラスラベルから整数に変換
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

fit_transformはfitとtransformを別々に呼び出すことに相当するショートカット。  
もとに戻すにはinverse_transformメソッドを使用する

In [19]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### 4.2.4 名義特徴量でのone-hotエンコーディング
* クラスラベルは「順序を持たないカテゴリデータ」として扱うため、単純に文字列のラベルを整数に変換した

* Tシャツのcolorについても同じようにマッピングできると思う。思うよね？

In [20]:
# Tシャツの色、サイズ、価格を抽出
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, '10.1'],
       [2, 2, '13.5'],
       [0, 3, '15.3']], dtype=object)

* 1列目のcolor列が整数に変換された
    
    * blue ⇒　0
    * green ⇒　1
    * red ⇒　2
    
    
* green, red, blueの間に大小関係が生まれてしまった。
* 学習アルゴリズムがこの大小関係を想定した結果を出す可能性がある。
* この問題を解決するために**one-hotエンコーディング**という手法をつかう

* one-hotエンコーディングとは
    * 名義特徴量の列の一意な値ごとにダミー特徴量を新たに作成する
    * scikit-learn.preprocessingモジュールで実装されているOneHotEncoderクラスで可能。

In [29]:
from sklearn.preprocessing import OneHotEncoder
# one-hotエンコーダの生成
ohe = OneHotEncoder(categorical_features=[0]) # 変換したい列の位置をリストで定義 #ここではX
# one-hotエンコードを実行
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

* toarray()メソッドを使うことで疎行列ではなく密行列であるNumPy配列に変換している。  
* toarrayはOneHotEncoder(..., sparse=Flase)でOK.  

* pandasのget_dummies関数の方が便利。

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
color         3 non-null object
size          3 non-null int64
price         3 non-null object
classlabel    3 non-null object
dtypes: int64(1), object(3)
memory usage: 176.0+ bytes


In [47]:
df['price']=df['price'].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
color         3 non-null object
size          3 non-null int64
price         3 non-null float64
classlabel    3 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 176.0+ bytes


In [49]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


_**多重共線性(multicollinearity)**_とは  
各特徴量間の相関が高い場合に正しく推計できなるような悪影響を及ぼす。  
特徴量間の相関性を減らすには、one-hotエンコーディングの配列から特徴量の列を1つ削除すればよい。  
例えば、color_blueを削除しても、color_green=0とcolor_red=0が観測されれば、その観測地がblueであることは自ずとわかる。  

get_dummies関数では、drop_first=Trueを渡すことで最初の列を削除することができる。

In [50]:
# one-hotエンコーディングを実行
pd.get_dummies(df[['price', 'size', 'color']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [52]:
#OneHotEncoderでもNumPy配列から列を削除するのは簡単
ohe = OneHotEncoder(categorical_features=[0], sparse=False)

# 実行
ohe.fit_transform(X)[:, 1:]

array([[  1. ,   0. ,   1. ,  10.1],
       [  0. ,   1. ,   2. ,  13.5],
       [  0. ,   0. ,   3. ,  15.3]])

## 4.3 データセットをトレーニングデータセットとテストデータセットに分割する。
**Wine**というデータセットに前処理を行い、次元数を減らすための特徴選択の手法を学ぶ。

In [54]:
# wineデータセットを読み込む
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)

In [58]:
# 列名を設定
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids',
                   'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

#クラスラベルを表示
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


* このデータセットはクラス1,2,3のいずれかに属することがわかった。
* イタリアの同じ地域で栽培されている異なる品種のブドウをあらわしている。

* train_test_split関数でランダムにトレーニングデータとテストデータに分割できる。

In [59]:
from sklearn.model_selection import train_test_split

# 特徴量とクラスラベルを別々に抽出
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 全体の30％をテストデータにする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

_**stratify=y**_を指定することでトレーニングセットとテストセットのクラスの比率が元のデータセットと同じになるようにしている。

## 4.4 特徴量の尺度を揃える
* 特徴量スケーリング(feature scaling)
    * 正規化(normalization)　⇒　特徴量を[0,1]の範囲にスケーリングしなおすことを意味する。min-maxスケーリング。
    * 標準化(standardization) ⇒　平均値0、標準偏差1になるように変換する。特徴量を正規分布に従わせる。外れ値に関する有益な情報が維持される。

In [60]:
# 正規化
# min-maxスケーリング
from sklearn.preprocessing import MinMaxScaler
# min-maxスケーリングのインスタンスを生成
mms = MinMaxScaler()

# トレーニングデータをスケーリング
X_trian_norm = mms.fit_transform(X_train)

# テストデータをスケーリング
X_test_norm = mms.fit_transform(X_test)

In [61]:
# 標準化と正規化を実行する
ex = np.array([0,1,2,3,4,5])
print('standardization:', (ex-ex.mean())/ex.std())

standardization: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]


In [62]:
print('normalization:', (ex-ex.min())/(ex.max()-ex.min()))

normalization: [ 0.   0.2  0.4  0.6  0.8  1. ]


In [66]:
# 標準化はsklearnでもできる
from sklearn.preprocessing import StandardScaler

#標準化のインスタンスを作成
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.fit_transform(X_test)

#print(X_train_std)
#print(X_test_std)

## 4.5 有益な特徴量の選択